In [1]:
%matplotlib notebook
from __future__ import print_function

import sys
import numpy as np
import xml.etree.ElementTree as ET
import logging
import glob
import matplotlib.pyplot as plt
import os

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from lmfit import Model
from lmfit.models import LinearModel, StepModel

In [2]:
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

plt.rcParams['figure.figsize'] = [8, 6]

In [3]:
# Constants
barscan_files = '/HFIR/CG2/IPTS-828/exp331/Datafiles/CG2_exp331_scan0005_0*.xml'
# chop the ends of the tube
tube_cut_bottom = 10
tube_cut_top = 10


In [4]:
def process(filename, xpath="Data/Detector"):

    # logger.info("Parsing: %s." % filename)
    tree = ET.parse(filename)
    root = tree.getroot()

    def getMetadata(xpath):
        '''
        Given Xpath returns either float or string
        '''
        elems = root.findall(xpath)
        if not elems:
            logger.error("xpath %s is not valid!" % xpath)
            return None
        elif len(elems) > 1:
            logger.warning("xpath %s has more than one element (len = %d)! Returning first!" % (
                xpath, len(elems)))
        value_as_string = elems[0].text
        try:
            return float(value_as_string)
        except ValueError:
            return value_as_string

    def getData(xpath):
        '''
        Parses the XML xpath data into a 2D Xarray
        '''
        data_str = getMetadata(xpath)
        data_list_of_chars = [line.split("\t")
                              for line in data_str.strip().split("\n")]
        data = [list(map(int, line)) for line in data_list_of_chars]
        data_np = np.array(data)
        data_np = np.rot90(data_np)
        data_np = np.flipud(data_np)
        return data_np

    data = getData(xpath)
    return data

## Plot detector

In [5]:
def show_detector(filename):
    data = process(filename, xpath="Data/Detector")
    fig, ax = plt.subplots()
    im = ax.pcolormesh(data)
    fig.colorbar(im, ax=ax)
    ax.set_title(os.path.basename(filename))
    ax.set_xlabel('Tube')
    ax.set_ylabel('Pixel')
    fig.canvas.draw()

In [6]:
files = glob.glob(barscan_files)
logger.debug("Read {} files.".format(len(files)))

# Natural sorting the files
files = sorted(files, key=lambda name: int(name[-8:-4]))

DEBUG:Read 111 files.


In [7]:
# Convert from integer to filenames
def show_detector_integer(point_number):
    show_detector('/HFIR/CG2/IPTS-828/exp331/Datafiles/CG2_exp331_scan0005_{0:04d}.xml'.format(point_number))

interact(show_detector_integer, point_number=(1, len(files)))


<function __main__.show_detector_integer>

## Plot tubes

In [8]:
def fit(x, y, e):
    
    step_mod = StepModel(form='atan', prefix='step_')
    line_mod = LinearModel(prefix='line_')

    pars = line_mod.make_params(intercept=y.min(), slope=0)
    pars += step_mod.guess(y, x=x, center=0.2)

    mod = step_mod + line_mod
    out = mod.fit(y, pars, x=x)#, weights=1/e)
    
    return out
    

In [9]:

def show_tube(filename, tube=0):

    x = np.linspace(-0.54825, 0.54825, 256)
    x = x[tube_cut_bottom:-tube_cut_top]
    
    data = process(filename, xpath="Data/Detector")
    tube_to_plot = data[tube_cut_bottom:-tube_cut_top, tube]

    fig, ax = plt.subplots()
    ax.errorbar(x, tube_to_plot, yerr=np.sqrt(tube_to_plot), fmt='x', capthick=4, label="Tube")
    

    min_pos = np.argmin(tube_to_plot)
    
    x = x[min_pos-10:min_pos+5]
    y = tube_to_plot[min_pos-10:min_pos+5]
    
    fit_out = fit(x, y, e=np.sqrt(y))
    print(fit_out.fit_report())
#     print(dir(fit_out))
#     print(fit_out.best_values['step_center'])
    
    ax.plot(x, fit_out.best_fit, label="Fit")
    
    ax.set_title("{} :: Tube={}".format(os.path.basename(filename), tube))
    ax.set_xlabel('Pixel')
    ax.set_ylabel('Counts')
    ax.grid(True)
    ax.legend()
    fig.canvas.draw()

In [10]:
interact(show_tube, filename=files, tube=(0, 191))

<function __main__.show_tube>